In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 04 — Register & Deploy Model
# MAGIC 
# MAGIC This notebook:
# MAGIC - Retrieves the latest MLflow run from the training experiment
# MAGIC - Registers the model in the Databricks Model Registry
# MAGIC - Transitions the model to **Staging** (or Production)
# MAGIC - Prepares the model for batch or real‑time deployment
# MAGIC 
# MAGIC This step completes the ML lifecycle and demonstrates production‑grade MLOps.

# COMMAND ----------


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Configuration

# COMMAND ----------

import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

model_name = "tpcds_customer_spend_model"
experiment_path = "/Shared/tpcds_ml_experiment"

print(f"Using experiment: {experiment_path}")
print(f"Model name: {model_name}")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Retrieve Latest MLflow Run

# COMMAND ----------

experiment = mlflow.get_experiment_by_name(experiment_path)
experiment_id = experiment.experiment_id

runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    order_by=["start_time DESC"],
    max_results=1
)

if runs.empty:
    raise ValueError("No MLflow runs found. Train a model first.")

latest_run = runs.iloc[0]
run_id = latest_run.run_id

print(f"Latest run ID: {run_id}")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Register Model in Model Registry

# COMMAND ----------

model_uri = f"runs:/{run_id}/model"

registered_model = mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)

print(f"Registered model version: {registered_model.version}")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Transition Model to Staging

# COMMAND ----------

client.transition_model_version_stage(
    name=model_name,
    version=registered_model.version,
    stage="Staging",
    archive_existing_versions=False
)

print(f"Model {model_name} v{registered_model.version} moved to Staging.")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Display Model Metadata

# COMMAND ----------

model_version_details = client.get_model_version(
    name=model_name,
    version=registered_model.version
)

print("Model Version Details:")
print(f"Name: {model_version_details.name}")
print(f"Version: {model_version_details.version}")
print(f"Stage: {model_version_details.current_stage}")
print(f"Run ID: {model_version_details.run_id}")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Optional: Enable Model Serving (UI Step)
# MAGIC 
# MAGIC To enable serving:
# MAGIC 1. Go to **Models** in the left sidebar.
# MAGIC 2. Select `tpcds_customer_spend_model`.
# MAGIC 3. Choose the version in **Staging**.
# MAGIC 4. Click **Enable Serving**.
# MAGIC 
# MAGIC This will create a REST endpoint for real‑time inference.


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC # Model Registration Complete
# MAGIC 
# MAGIC Your model is now:
# MAGIC - Logged in MLflow
# MAGIC - Registered in the Model Registry
# MAGIC - Transitioned to Staging
# MAGIC 
# MAGIC You can now use it for batch scoring or real‑time serving.
